# 📈 PyKwant Tutorial: Equity Options & The Greeks

This notebook introduces the **Equity** capabilities of PyKwant. We will model a European Option, price it using the **Black-Scholes** model, and compute its risk sensitivities (**The Greeks**) using Functional Differentiation.

## 1. Setup and Imports

In [3]:
from datetime import date
import math
from pykwant import instruments, rates, dates, equity, numerics

# Helper for display
def print_greek(label: str, value: float):
    print(f"{label:<10}: {value:>10.4f}")

## 2. Define the Option
Let's create a **Call Option** on "PYK Tech" with a Strike of 100, expiring in 1 year.

In [4]:
expiry = date(2026, 1, 1)
strike_price = instruments.Money(100.0)

call_option = instruments.EuropeanOption(
    asset_name="PYK Tech",
    strike=strike_price,
    expiry_date=expiry,
    type="call"
)

print(call_option)

EuropeanOption(asset_name='PYK Tech', strike=100.0, expiry_date=datetime.date(2026, 1, 1), type='call')


## 3. Market Environment
We need the current Spot Price, Volatility, and a Yield Curve.

In [5]:
valuation_date = date(2025, 1, 1)
spot_price = 100.0   # At the Money
volatility = 0.20    # 20%
risk_free_rate = 0.05 # 5%

# Create a flat yield curve function
def flat_curve(d: date) -> float:
    t = dates.act_365(valuation_date, d)
    return rates.compound_factor(-risk_free_rate, t, frequency=0)

## 4. Black-Scholes Pricing
We use the pure function `equity.black_scholes_price`.

In [6]:
price = equity.black_scholes_price(
    option=call_option,
    spot=spot_price,
    volatility=volatility,
    curve=flat_curve,
    valuation_date=valuation_date
)

print(f"Call Option Price: {price:.4f}")
# Expected approx 10.45 for S=100, K=100, T=1, r=5%, vol=20%

Call Option Price: 10.4506


## 5. Computing Greeks via Functional Differentiation
Instead of hardcoding formulas like $\Delta = N(d_1)$, we use `pykwant.numerics.numerical_derivative`. This approach is generic and works for any pricing model!
### Delta ($\partial P / \partial S$) & Gamma ($\partial^2 P / \partial S^2$)
We create a partial function `price_vs_spot(S)` and differentiate it.

In [7]:
# 1. Create a closure that depends only on Spot (S)
def price_vs_spot(s: float) -> float:
    return float(equity.black_scholes_price(
        call_option, s, volatility, flat_curve, valuation_date
    ))

# 2. Compute First Derivative (Delta)
delta_fn = numerics.numerical_derivative(price_vs_spot, h=0.01)
delta = delta_fn(spot_price)

# 3. Compute Second Derivative (Gamma)
gamma_fn = numerics.numerical_derivative(delta_fn, h=0.01)
gamma = gamma_fn(spot_price)

print("--- Spot Sensitivities ---")
print_greek("Delta", delta) # Expected ~0.63
print_greek("Gamma", gamma) # Expected ~0.018

--- Spot Sensitivities ---
Delta     :     0.6368
Gamma     :     0.0188


### Vega ($\partial P / \partial \sigma$)
Sensitivity to Volatility.

In [8]:
# 1. Closure depending on Vol (sigma)
def price_vs_vol(v: float) -> float:
    return float(equity.black_scholes_price(
        call_option, spot_price, v, flat_curve, valuation_date
    ))

# 2. Derivative
vega_fn = numerics.numerical_derivative(price_vs_vol, h=1e-4)
vega = vega_fn(volatility)

# Usually quoted for 1% change, so divide by 100
print("\n--- Volatility Sensitivity ---")
print_greek("Vega", vega)       # Raw derivative
print_greek("Vega (1%)", vega * 0.01)


--- Volatility Sensitivity ---
Vega      :    37.5240
Vega (1%) :     0.3752


### Theta ($\partial P / \partial t$)
Sensitivity to Time Decay (negative). We move the valuation date forward by 1 day.

In [9]:
# Theta is easier to calculate via finite difference directly on dates
from datetime import timedelta

price_tomorrow = equity.black_scholes_price(
    call_option, spot_price, volatility, flat_curve, valuation_date + timedelta(days=1)
)

theta_1d = price_tomorrow - price

print("\n--- Time Sensitivity ---")
print_greek("Theta (1d)", theta_1d)
# Annualized Theta
print_greek("Theta (1y)", theta_1d * 365)


--- Time Sensitivity ---
Theta (1d):    -0.0103
Theta (1y):    -3.7548


## 6. Put-Call Parity Check
Functional programming makes verification easy. Let's check: $C - P = S - K \cdot e^{-rT}$.

In [10]:
# Create Put
put_option = instruments.EuropeanOption(
    "PYK Tech", strike_price, expiry, "put"
)

put_price = equity.black_scholes_price(
    put_option, spot_price, volatility, flat_curve, valuation_date
)

# LHS
lhs = price - put_price

# RHS
df = flat_curve(expiry)
rhs = spot_price - strike_price * df

print("\n--- Put-Call Parity Check ---")
print(f"Call - Put        : {lhs:.6f}")
print(f"Spot - PV(Strike) : {rhs:.6f}")
assert math.isclose(lhs, rhs, abs_tol=1e-5)
print("Parity Holds! ✅")


--- Put-Call Parity Check ---
Call - Put        : 4.877058
Spot - PV(Strike) : 4.877058
Parity Holds! ✅
